<a href="https://colab.research.google.com/github/kirath2205/Final-Year-Project/blob/main/Inception.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np


def get_random_eraser(p=0.5, s_l=0.02, s_h=0.4, r_1=0.3, r_2=1/0.3, v_l=0, v_h=255, pixel_level=False):
    def eraser(input_img):
        img_h, img_w, img_c = input_img.shape
        p_1 = np.random.rand()

        if p_1 > p:
            return input_img

        while True:
            s = np.random.uniform(s_l, s_h) * img_h * img_w
            r = np.random.uniform(r_1, r_2)
            w = int(np.sqrt(s / r))
            h = int(np.sqrt(s * r))
            left = np.random.randint(0, img_w)
            top = np.random.randint(0, img_h)

            if left + w <= img_w and top + h <= img_h:
                break

        if pixel_level:
            c = np.random.uniform(v_l, v_h, (h, w, img_c))
        else:
            c = np.random.uniform(v_l, v_h)

        input_img[top:top + h, left:left + w, :] = c

        return input_img

    return eraser

In [2]:
!pip install Keras-applications

     |████████████████████████████████| 50 kB 2.7 MB/s 


In [10]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
import tensorflow.keras as keras
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, UpSampling2D, Flatten, BatchNormalization, Dense, Dropout, GlobalAveragePooling2D , UpSampling3D
from tensorflow.keras import optimizers
from keras.datasets import cifar100,cifar10,fashion_mnist
import tensorflow as tf
from keras.utils import np_utils
import numpy as np
import matplotlib.pyplot as plt
import time
from skimage.transform import resize
from keras import applications
from tensorflow.keras.applications.resnet50 import preprocess_input , decode_predictions
from keras.preprocessing.image import ImageDataGenerator

In [11]:
def select_dataset(index=1): #1 for cifar10 , 2 for cifar100 , 3 for fashion mnist
  if(index == 1):
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()
    channel = 3
    num_classes = 10
  if(index == 2):
    (x_train, y_train), (x_test, y_test) = cifar100.load_data()
    num_classes = 100
    channel = 3
  if(index == 3):
    (x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
    x_train = x_train.reshape((60000, 28, 28, 1))
    x_test =  x_test.reshape((10000, 28, 28, 1))
    num_classes = 10
    y_train = np_utils.to_categorical(y_train, num_classes)
    y_test = np_utils.to_categorical(y_test, num_classes)
    channel = 1
    return (x_train , y_train , x_test , y_test , num_classes , channel)

  #Pre-process the data
  x_train = preprocess_input(x_train)
  x_test = preprocess_input(x_test)

  datagen = ImageDataGenerator(preprocessing_function=get_random_eraser(v_l=0, v_h=1, pixel_level=True))
  datagen.fit(x_train)
  y_train = np_utils.to_categorical(y_train, num_classes)
  y_test = np_utils.to_categorical(y_test, num_classes)

  return (x_train , y_train , x_test , y_test , num_classes , channel , datagen)

In [18]:
def Inception(num_classes , channel=3):
  if(channel == 3):
    inception_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(160, 160, 3))
  else:
    inception_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(112, 112, 3))

  for layer in inception_model.layers:
    if isinstance(layer, BatchNormalization):
      layer.trainable = True
    else:
      layer.trainable = False
  model = Sequential()
  if(channel==1):
    model.add(UpSampling3D((4,4,3)))
  else:
    model.add(UpSampling2D((5,5)))
  model.add(inception_model)
  model.add(GlobalAveragePooling2D())
  model.add(Dense(1024, activation='relu'))
  model.add(Dropout(.25))
  #model.add(BatchNormalization())
  model.add(Dense(512, activation='relu'))
  model.add(Dropout(.25))
  #model.add(BatchNormalization())
  model.add(Dense(256, activation='relu'))
  #model.add(Dropout(.25))
  model.add(BatchNormalization())
  model.add(Dense(num_classes, activation='softmax'))
  model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
  return model

In [ ]:
index = 1
x_train , y_train , x_test , y_test , num_classes ,channel , datagen = select_dataset(index)
model = Inception(num_classes,channel)
model_name = 'inception_cifar10'
model_path = '/content/inception_cifar10.h5'

In [19]:
index = 2
x_train , y_train , x_test , y_test , num_classes ,channel , datagen = select_dataset(index)
model = Inception(num_classes,channel)
model_name = 'inception_cifar100'
model_path = '/content/inception_cifar100.h5'

In [ ]:
index = 3
x_train , y_train , x_test , y_test , num_classes ,channel  = select_dataset(index)
model = Inception(num_classes,channel)
model_name = 'inception_mnist'
model_path = '/content/inception_mnist.h5'

In [20]:
batch_size = 128
callbacks = [ 
    tf.keras.callbacks.ModelCheckpoint(model_path, monitor='val_accuracy', verbose=1, save_best_only=True),
    tf.keras.callbacks.ReduceLROnPlateau( factor = 0.5, patience = 3, min_lr = 0.000001, verbose = 1 ),
    tf.keras.callbacks.EarlyStopping(monitor = 'val_loss' , patience = 5)
  ]
if(channel == 3):
  history = model.fit_generator(datagen.flow(x_train, y_train,
                                  batch_size=batch_size),
                                  steps_per_epoch=x_train.shape[0] // batch_size,
                                  epochs=50,
                                  validation_data=(x_test, y_test),
                                  callbacks = callbacks)

  model.save(model_path)
else:
  history = model.fit(x_train , y_train , batch_size=batch_size ,steps_per_epoch=x_train.shape[0] // batch_size, epochs=50, validation_data=(x_test, y_test),callbacks=callbacks)
  model.save(model_path)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  del sys.path[0]


Epoch 1/50
390/390 [==============================] - ETA: 0s - loss: 3.6740 - accuracy: 0.1635
Epoch 00001: val_accuracy improved from -inf to 0.37660, saving model to /content/inception_cifar100.h5
390/390 [==============================] - 327s 806ms/step - loss: 3.6740 - accuracy: 0.1635 - val_loss: 2.3522 - val_accuracy: 0.3766 - lr: 0.0010
Epoch 2/50
390/390 [==============================] - ETA: 0s - loss: 2.6588 - accuracy: 0.3218
Epoch 00002: val_accuracy improved from 0.37660 to 0.47640, saving model to /content/inception_cifar100.h5
390/390 [==============================] - 312s 801ms/step - loss: 2.6588 - accuracy: 0.3218 - val_loss: 1.9343 - val_accuracy: 0.4764 - lr: 0.0010
Epoch 3/50
390/390 [==============================] - ETA: 0s - loss: 2.3023 - accuracy: 0.3978
Epoch 00003: val_accuracy improved from 0.47640 to 0.52380, saving model to /content/inception_cifar100.h5
390/390 [==============================] - 312s 800ms/step - loss: 2.3023 - accuracy: 0.3978 - val